# Bayesian optimization of a time-consuming simulator

The aim of the lab is to optimize a time-consuming simulator using the Efficient Global Optimization (EGO) method. As a toy example, the simulator chosen here mimics a catapult. There are 4 input variables, tuning the catapult, and 1 output, giving the distance of the projectile mark to the catapult. We want to find the value(s) of the input variable(s) maximizing that distance. Notice that the simulator is not time-consuming. However, for the sake of realism, we will limit the computational budget to 36 runs.

1. "By-hand" Optimization
<br> We provide here a shiny application (authored by Nicolas Durrande), which allows using the simulator interactively.

In [ ]:
library(shiny)
runApp()

*Question: By playing with the simulator, propose a set of input values giving the largest possible value of the output.*

To continue running the notebook, you may need to interrupt the kernel (square symbol!). Then load the two following scripts, containing useful functions. 

In [ ]:
source("catapultSettings.R")
source("catapultFunctions.R")

2. Let us create an initial design of experiences and compute the corresponding values.

In [ ]:
library(DiceDesign)
X0 <- lhsDesign(n = 16, dimension = 4)$design
Xopt <- maximinESE_LHS(X0, it=10)
## you may be interested in the convergence
#plot(Xopt$critValues,type="l")
X <- Xopt$design
colnames(X) <- c("axe", "butee", "ressort1", "ressort2")
pairs(X)
## compute the output values
Y <- apply(X, 1, runExperiment)[1, ]

Question : Observe that the design of experiments is "space-filling". <br> Why did we chose that kind of designs? What is the current maximum? Is it far from the maximum value found by-hand?

3. Descriptive statistics. 
<br> *Question : Can you see a simple input-output relation? What can you say about the area corresponding to the maximum value?*

In [ ]:
pairs(cbind(Y, X))

4. Regression metamodel.
<br> *Question : Try the linear models below. If you replace the simulator by one of this model, what would be the optimum? Is it far from your previous guess?*

In [ ]:
myData <- data.frame(X, Y = Y)
m <- lm(Y ~ ., data = myData)
summary(m)
m2 <- lm(Y~. + I(axe^2) + I (butee^2) + I (ressort1^2) + I(ressort2^2), data = myData)
summary(m2)
mstep <- step(object = m, scope = m2, direction = "both", k = log(length(Y)))
summary(mstep)

5. Now, let us try the EGO method (Bayesian optimization)
<br> *Question : Recall its main principles.*

In [ ]:
library(DiceKriging)
m0 <- km(~1, design=X, response=Y)
print(m0)   # display model
plot(m0)    # visual model validation

*Question : What can you say of this first model? Comment the estimated value of lengthscale parameters: Are they consistent to the results of the linear model?*

In [ ]:
library(DiceOptim)

## N.B. We first transform the optimization problem in a maximization problem 
runExperimentMin <- function(x) {
    - apply(trajectory(x), 2, max)[1]
    }
Y <- apply(X, 1, runExperimentMin)
m0 <- km(~1, design = X, response = Y)

## Step 1 ##
oEGO <- max_EI(model = m0, lower = rep(0, 4), upper = rep(1, 4))
newX <- oEGO$par
newy <- runExperimentMin(newX)

cat("Expected improvement was :", round(oEGO$value,2),
    "\nActual improvement is:", round(min(Y) - newy,2),
    "\n   (>0 means the new point is better, <0 means its worst)")

In [ ]:
# Then we update the model
m1 <- update(m0, newX, newy)
# and maximize again the expected improvement: 
## Step 2 ##
oEGO <- max_EI(model = m1, lower = rep(0, 4), upper = rep(1, 4))
newX <- oEGO$par
newy <- runExperimentMin(newX)

cat("Expected improvement was :", round(oEGO$value,2),
    "\nActual improvement is:", round(min(Y) - newy,2),
    "\n   (>0 means the new point is better, <0 means its worst)")

In [ ]:
# Fortunately, the loop (model update -> EI maximisation -> ...) is already implemented
oEGO <- EGO.nsteps(model = m0, fun = runExperimentMin, nsteps = 20, 
                   lower = rep(0, 4), upper = rep(1, 4))

bestPoint <- which.min(oEGO$value)
cat("longest shot observed:",-round(oEGO$value[bestPoint],2),
    "\ncorresponding input values:",round(oEGO$par[bestPoint,],2))


Let us visualize the 20 points computed with EGO in the X-Y space, and in time order.

In [ ]:
visualizeEGO <- function(initDesign, initValues, EGOpoints, EGOvalues){
  bestIndex <- which.min(EGOvalues)
  y <- c(initValues, EGOvalues, EGOvalues[bestIndex])
  X <- rbind(initDesign, EGOpoints, EGOpoints[bestIndex, ])
  ninit <- nrow(initDesign)
  nsteps <- nrow(EGOpoints)
  pairs(cbind(y, X), 
        col = c(rep("black", ninit), rep("blue", nsteps), "red"),
        pch = c(rep(1, ninit + nsteps), 19))
}

visualizeEGO(initDesign = X, initValues = Y,
             EGOpoints = oEGO$par, EGOvalues = oEGO$val)

In [ ]:
plot(c(Y, oEGO$value), main="convergence", 
     xlab="evaluation number", ylab="Y values")
lines(rep(length(Y), 2), range(Y, oEGO$value), lty = 2, col = "gray")
lines(length(Y) + 0:length(oEGO$value), c(min(Y), cummin(oEGO$value)), col="red", lwd=2)

 Questions : 
 * *Why the EGO method would be much less efficient by using a linear model instead of a GP model?* 
 * *Modify the code in order to investigate the influence of a trend in the model (change formula in 'km'), of a kernel (?km), of the initial sample size.*
 * *Adapt the EGO method in order to provide a batch of 2 points at one (function qEGO.nsteps), which is useful in practice, as the 2 runs of the time-consuming simulator can be done in parallel.*

Bonus : As the function is quick to evaluate, compare the result with the maximum value obtained with a standard optimizor (from library nloptr for instance). You can use the following function, which avoids the graphical representation of the catapult.

In [ ]:
myFun <- function(x) apply(trajectory(x), 2, max)[1]
myFun(c(0.5, 0.5, 0.5, 0.5))